# Task 0:  Installs and Environment Variables 

In [23]:
# Uninstall incompatible version of langchain-core as a caution
%pip uninstall langchain-core -y

# Install compatible version of langchain-core
%pip install langchain-core==0.2.40 langchain-huggingface==0.0.3 ragas==0.1.20

# Check for any remaining package conflicts
%pip check


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: langchain-core 0.2.40
Uninstalling langchain-core-0.2.40:
  Successfully uninstalled langchain-core-0.2.40
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_core-0.2.40-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.40-py3-none-any.whl (396 kB)
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


No broken requirements found.
Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

# Task 1:  Dealing with the Data

In [25]:
from langchain_community.document_loaders import PyMuPDFLoader

# List of file paths for the PDFs you want to load
paths = ["https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf", "https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf"]

# Create a list to store loaded documents
documents = []

# Loop through each PDF and load it
for path in paths:
    loader = PyMuPDFLoader(path)
    documents.extend(loader.load())  # Add the documents to the list

len(documents)

137

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

chunks = text_splitter.split_documents(documents)
len(chunks)

2898

In [27]:
# from langchain_openai import OpenAIEmbeddings

# # Specify the embedding model
# EMBEDDING_MODEL = "text-embedding-ada-002"

# # Initialize the OpenAIEmbeddings class
# embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

In [28]:
from langchain_huggingface import HuggingFaceEmbeddings

# Specify the open-source embedding model from Hugging Face
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize the HuggingFaceEmbeddings class with the chosen model
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

# Example
# text = "This is an example sentence for generating embeddings."
# embedding_vector = embeddings.embed_query(text)
# print(embedding_vector)

/opt/anaconda3/envs/aie4/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

LOCATION = ":memory:"
COLLECTION_NAME = "Midterm"
VECTOR_SIZE = 384

# Initialize the Qdrant client
qdrant_client = QdrantClient(
    location=LOCATION
    )

# Create a collection in Qdrant
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=VECTOR_SIZE, 
        distance=Distance.COSINE
        )
    )

# Initialize QdrantVectorStore with the Qdrant client
qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
)

# Add documents to the collection
qdrant_vector_store.add_documents(documents)

['6315bec2e1204ff38c865ebf1faf575d',
 '21400c4bb76c40019714161be617bdea',
 '8075057128974112bb907b9039620a54',
 'b24a4a4ac7ff4e389f58e256904023d8',
 'bb40ff277b044edea00c3d8e25a260dc',
 'fabf8bdfb9f34deab2ccdca5126c1c90',
 '25983f9a0d1d41b6a793c3158cbb5aee',
 '36bc7c33c08d45049313f08ddac4875b',
 '8a55862d92b640409ef644696003b353',
 '7bf4b75ca93648459a3cbcb7b49fe4c0',
 '47fd0b83986e4830997d2a8245c1f35a',
 'b3b0ffe317cd4804a8860d6dfcc330ad',
 'd1ee2a365aaf49c0923f0d6794eabe99',
 '5442a8cb074d4671abb3001bf11e3e90',
 '9e157d370c7a4209b10eb73618b392b5',
 '794819f83eaa4e10b2de05c2852396f3',
 '730a926d1d424c878eea157f957972df',
 '9c290fae24274c45abf674b41cd5e5c0',
 'e5441b2bb2754146a53352f68dd81f52',
 '160325497e6047a2a351a0ffaca317f1',
 '510bc7029fb44ceab1b41d052c99324c',
 '1e30cf5ed37345199bc156de535bd02b',
 'e033b64472a6481392d0cf2a312c0ea3',
 '4f27770800424090a946e706acc22092',
 '4d42841014fa4999af77d39ed4a49df9',
 '1c8c98ec60254114b9d66eed2a116300',
 'be4c7bc5d241432ea26bbd467f70a548',
 

In [30]:
retriever = qdrant_vector_store.as_retriever()

# retrieved_documents = retriever.invoke("What are underserved communities?")
# retrieved_documents = retriever.invoke("What should be expected of automated systems?")
retrieved_documents = retriever.invoke("What is action ID GV-1.3-001?")

for doc in retrieved_documents:
  print(doc)

page_content=' 
19 
GV-4.1-003 
Establish policies, procedures, and processes for oversight functions (e.g., senior 
leadership, legal, compliance, including internal evaluation) across the GAI 
lifecycle, from problem formulation and supply chains to system decommission. 
Value Chain and Component 
Integration 
AI Actor Tasks: AI Deployment, AI Design, AI Development, Operation and Monitoring 
 
GOVERN 4.2: Organizational teams document the risks and potential impacts of the AI technology they design, develop, deploy, 
evaluate, and use, and they communicate about the impacts more broadly. 
Action ID 
Suggested Action 
GAI Risks 
GV-4.2-001 
Establish terms of use and terms of service for GAI systems. 
Intellectual Property; Dangerous, 
Violent, or Hateful Content; 
Obscene, Degrading, and/or 
Abusive Content 
GV-4.2-002 
Include relevant AI Actors in the GAI system risk identiﬁcation process. 
Human-AI Conﬁguration 
GV-4.2-003 
Verify that downstream GAI system impacts (such as the u

# Task 2:  Building a Quick End-to-End Prototype

In [31]:
from langchain.prompts import ChatPromptTemplate

template = """
Only answer the question using the context below.  If the answer can't be found in the context, respond "I don't know". 

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from operator import itemgetter

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
    | RunnablePassthrough.assign(context=itemgetter("context")) 
    | {"response": prompt | llm, "context": itemgetter("context")}
    )  

In [35]:
question = "What is confabulation?"
#question = "What is action ID GV-1.3-002?"

result = chain.invoke({"question" : question})

print(result["response"].content)

Confabulation refers to a phenomenon in which GAI systems generate and confidently present erroneous or false content in response to prompts. It includes generated outputs that diverge from the prompts or contradict previously generated statements in the same context. These phenomena are also known as "hallucinations" or "fabrications."


# Creating a Golden Test Data Set